In [10]:
import torch.utils.data
import sklearn.metrics
import torch
import pandas
from torch.utils.data import Dataset
import tqdm
import spacy
import numpy as np
nlp = spacy.load('en')
pandas.read_csv('sentiment.tsv',sep='\t',header=0).iloc[0]

id                                                      5814_8
sentiment                                                    1
review       With all this stuff going down at the moment w...
Name: 0, dtype: object

In [11]:
class SentimentDataset(Dataset):
    def __init__(self):
        self.data = pandas\
            .read_csv('sentiment.tsv',sep='\t',header=0)\
            .groupby('id')\
            .first()
            
            
        self.ordinals = {}
        for sample in tqdm.tqdm(self.data.review):
            for token in nlp (sample.lower(),disable = ['parser','tagger','ner']):
                if token.text not in self.ordinals:
                    self.ordinals[token.text] = len(self.ordinals)
    def __len__ (self):
        return len(self.data)
    def __getitem__(self,idx):# idx increase from 0 to len of data __len__ ()
        if type(idx) is torch.Tensor:
            idx = idx.item()
        sample = self.data.iloc[idx]
        bag_of_words = torch.zeros(len(self.ordinals))
        for token in nlp (sample.review.lower(),disable = ['parser','tagger','ner']):
            bag_of_words[self.ordinals[token.text]] += 1
        return bag_of_words, torch.tensor(sample.sentiment)
ssentiment = SentimentDataset()
ssentiment[0]


100%|██████████| 25000/25000 [00:35<00:00, 700.55it/s]


(tensor([1., 3., 7.,  ..., 0., 0., 0.]), tensor(0))

In [12]:
number_for_testing = int(len(ssentiment)*0.05)
number_for_training = len(ssentiment) - number_for_testing
test,train = torch.utils.data.random_split(ssentiment,[
    number_for_testing,number_for_training])
trainloader = torch.utils.data.DataLoader(train,batch_size =32, shuffle =True)
testloader = torch.utils.data.DataLoader(test,batch_size =32, shuffle =True)
len(test),len(train)
len(trainloader)
ssentiment.data.sentiment.unique()

array([0, 1])

In [0]:
class Modelsentiment (torch.nn.Module):
    def __init__(self, input_dimensions,size=128):
        super().__init__()
        self.layer_one = torch.nn.Linear(input_dimensions, size)
        self.activation_one = torch.nn.ReLU()
        self.layer_two = torch.nn.Linear(size, size)
        self.activation_two = torch.nn.ReLU()
        self.shape_outputs = torch.nn.Linear(size, 2)

    def forward(self, inputs):

        buffer = self.layer_one(inputs)
        buffer = self.activation_one(buffer)
        buffer = self.layer_two(buffer)
        buffer = self.activation_two(buffer)
        buffer = self.shape_outputs(buffer)
        return buffer
modelsentiment = Modelsentiment(len(ssentiment.ordinals))

In [14]:
optimizer = torch.optim.Adam(modelsentiment.parameters())
loss_function = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(modelsentiment.parameters())
loss_function = torch.nn.CrossEntropyLoss()
modelsentiment.train()
for epoch in range(4):
    losses = []
    for inputs, outputs in trainloader:
        optimizer.zero_grad()
        #print(inputs.size())
        results = modelsentiment(inputs)
        loss = loss_function(results, outputs)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("Loss: {0}".format(torch.tensor(losses).mean()))



Loss: 0.3264491558074951
Loss: 0.09967786818742752
Loss: 0.04260505363345146
Loss: 0.027972184121608734


In [15]:
results_buffer = []
actual_buffer = []
with torch.no_grad():
    modelsentiment.eval()

    for inputs, actual in testloader:
        results = modelsentiment(inputs).argmax(dim=1).numpy()
        results_buffer.append(results)
        actual_buffer.append(actual)
print(sklearn.metrics.classification_report(
    np.concatenate(actual_buffer),
    np.concatenate(results_buffer)))

              precision    recall  f1-score   support

           0       0.88      0.87      0.87       619
           1       0.88      0.88      0.88       631

    accuracy                           0.88      1250
   macro avg       0.88      0.88      0.88      1250
weighted avg       0.88      0.88      0.88      1250

